## Train the neural network / random forest and run it on testing data

## Test that I am extracting the parameters correctly

In [ ]:
out_test = r_mlp.predict(x3)
out_test = scaler_y.inverse_transform(out_test)
w1 = r_mlp.get_parameters()[0].weights
w2 = r_mlp.get_parameters()[1].weights
w3 = r_mlp.get_parameters()[2].weights
b1 = r_mlp.get_parameters()[0].biases
b2 = r_mlp.get_parameters()[1].biases
b3 = r_mlp.get_parameters()[2].biases

xscale_min = scaler_x.data_min_
xscale_max = scaler_x.data_max_
yscale_absmax = scaler_y.max_abs_

out_test_check = np.dot(x3,w1) + b1
out_test_check[out_test_check<0] = 0
out_test_check = np.dot(out_test_check,w2) + b2
out_test_check[out_test_check<0] = 0
out_test_check = np.dot(out_test_check,w3) + b3

out_test_check = out_test_check*yscale_absmax
ptl.plot(out_test-out_test_check)
plt.show()

### Check that I understand what classification is doing

In [ ]:

c_w1=c_mlp.get_parameters()[0].weights
c_w2=c_mlp.get_parameters()[1].weights
c_w3=c_mlp.get_parameters()[2].weights
c_b1=c_mlp.get_parameters()[0].biases
c_b2=c_mlp.get_parameters()[1].biases
c_b3=c_mlp.get_parameters()[2].biases

out_test_check = np.dot(x1            ,c_w1) + c_b1
out_test_check[out_test_check<0] = 0
out_test_check = np.dot(out_test_check,c_w2) + c_b2
out_test_check[out_test_check<0] = 0
out_test_check = np.dot(out_test_check,c_w3) + c_b3

expo =  np.exp(out_test_check)
expos = np.sum(expo,axis=1)

#foo=np.empty((x1.shape[0], 2))
foo[:,0] = expo[:,0]/expos
foo[:,1] = expo[:,1]/expos
ff=np.zeros(x1.shape[0])
ff[foo[:,1]>0.5]=1.
print(x1.shape)
ee=c_mlp.predict(x1)
ee=np.squeeze(ee)
print(ee.shape)
print(ff.shape)
print(np.sum(np.logical_and(ff==0,ee==1)))
print(np.sum(np.logical_and(ff==1,ee==0)))

rexpo = np.exp(out_test_check[:,1])

In [ ]:
plt.figure(figsize=(8,40))
_,ax = plt.subplots(lev.size,2,sharex=True)
for i in range(lev.size):
    step=.05
    bins=np.arange(-1,1+step,step)
    n,bins,_ =ax[i,0].hist(unpack(x_train_norm,'T')[:,i],bins=bins,facecolor='yellow',alpha=0.5,normed=True)
    n2,bins2,_=ax[i,1].hist(unpack(y_train_norm,'T')[:,i],bins=bins,facecolor='blue'  ,alpha=0.5,normed=True)


    ax[i,0].set_xlim((-1,1))
    ax[i,0].set_ylim(0,np.amax(n))
    ax[i,1].set_ylim(0,np.amax(n2))


    #ax[i,0].set_ylim([-1,1])
    print(np.amax(n))
    print(np.amax(n2))
    #ax[i,1].hist(unpack(x_train_norm,'q')[:,i]*step,bins=np.arange(-1,1+step,step),facecolor='yellow',alpha=0.5,normed=True)
    #ax[i,1].hist(unpack(y_train_norm,'q')[:,i]*step,bins=np.arange(-1,1+step,step),facecolor='blue'  ,alpha=0.5,normed=True)
    #ax[i,1].set_xlim([-1,1])



    #plt.subplot(lev.size,2,i+1+lev.size)
    #plt.hist(y_train_norm[:,i],100,facecolor='green')
    #ax[i,0].get_yaxis().set_visible(False)

    #n, bins, patches = plt.hist(y_train_norm[:,28], 100, normed=1, facecolor='green', alpha=0.75)
plt.show()

In [ ]:
# Train the neural network on the training data
hidden_neurons = 100
num_loops = 2#int(.1*np.power(10,6))
print(num_loops)
out_train_norm, model = nn2.run(x_train_norm, y_train_norm, hidden_neurons, print_loss=True, num_passes=num_loops,learning_rate=.01,regularization=.01)

# Run the (trained) neural network on the test data
out_test_norm, _ = nn2.fwd_prop(x_test_norm, model.w2, model.b2, model.w3, model.b3)

#Unnormalize model output
out_train = nn.unnorm(out_train_norm, ytrainmin, ytrainmax)
out_test  = nn.unnorm(out_test_norm,  ytestmin,  ytestmax )



In [ ]:
# Calculate out of bag error importance for random forest regressor


oob_mlp = RandomForestRegressor(n_estimators=30)

oob_mlp.fit(x2,y2)
indmin=np.argmin(oob_mlp.feature_importances_)
print(indmin)
print(np.min(oob_mlp.feature_importances_))
print(oob_mlp.score(x3,y3))
x2 = np.delete(x2,indmin,1)

plt.plot(unpack(oob_mlp.feature_importances_,'T'),lev,label='T')
plt.plot(unpack(oob_mlp.feature_importances_,'q'),lev,label='q')
plt.ylim((1,0))
plt.show()
oob_mlp2 = RandomForestRegressor(n_estimators=30)
oob_mlp2.fit(x2[:,oob_mlp.feature_importances_>0.],cv2)

#oob_mlp2.feature_importances_.shape
np.argmin(oob_mlp.feature_importances_)#[oob_mlp.feature_importances_>0.]

In [ ]:
do_nn_classifier = False
do_rf_classifier = False
do_randomforest  = False
do_neuralnetwork = True
do_my_nn         = False

errors = []
def store_stats( avg_train_error, **_):
    if np.mod(len(errors),100)==0:
        print("Epoch #%5d, Error=%.4f" %(len(errors),avg_train_error))
    errors.append(( avg_train_error))
    

# Preclassifier to determine if convection is occurring or not
if do_nn_classifier:
    classifier = []
    classifier = sknn.mlp.Classifier(layers=[sknn.mlp.Layer("Tanh",units=100),sknn.mlp.Layer("Tanh",units=100),sknn.mlp.Layer("Tanh",units=100),
                                             sknn.mlp.Layer("Softmax")],learning_rate=0.01,n_iter=10000,batch_size=100,learning_rule='momentum',n_stable=8000,
                                     callback={'on_epoch_finish': store_stats})
    classifier.fit(x1,y1cv)
    class_train=classifier.predict(x1)
    class_test=classifier.predict(x2)
    plt.plot(errors)
    plt.title('Error')
    plt.xlabel('Iteration #')
    plt.show()

if do_rf_classifier: #randomforest_classifier
    classifier = RandomForestClassifier(n_estimators=15)
    classifier.fit(x1,y1cv.ravel())
    class_train = classifier.predict(x1)
    class_test = classifier.predict(x2)
    # Plot feature importance
    plt.plot(classifier.feature_importances_[0:30],lev,label="T")
    plt.plot(classifier.feature_importances_[30:60],lev,label="q")
    plt.legend()
    plt.show()

# Random Forest
if do_randomforest:
    model = RandomForestRegressor(n_estimators=75)
    model.fit(x1,y1)
    out_train = model.predict(x1)
    out_test = model.predict(x2)
    # Plot feature importance
    plt.plot(model.feature_importances_[0:30],lev,label="T")
    plt.plot(model.feature_importances_[30:60],lev,label="q")
    plt.legend()
    plt.show()
    
# Neural Network through the lasagne package (via scikit-learn neural network)
if do_neuralnetwork:
    neur=[]
    neur=sknn.mlp.Regressor(layers=[sknn.mlp.Layer("Tanh",units=200),
                                    sknn.mlp.Layer("Tanh",units=200),sknn.mlp.Layer("Linear")],
                            n_iter=100,batch_size=100,callback={'on_epoch_finish': store_stats},
                            learning_rate=.01,learning_rule="momentum",learning_momentum=.7)
    neur.fit(x1,y1)
    out_train=neur.predict(x1)
    out_test=neur.predict(x2)
    plt.plot(errors)
    plt.title('Mean Squared Error')
    plt.xlabel('Iteration #')
    plt.show()

# Homemade neural network
if do_my_nn:
    hidden_neurons = 100
    num_loops = int(np.power(10,5))
    out_train, model = nn2.run(x1, y1, hidden_neurons, print_loss=True, num_passes=num_loops,learning_rate=.01,regularization=.01)
    # Run the (trained) neural network on the test data
    out_test, _ = nn2.fwd_prop(x2, model.w2, model.b2, model.w3, model.b3)

#Unnormalize model output
#out_train = y_scaler.inverse_transform(out_train)
#out_test =  y_scaler.inverse_transform(out_test )

